## Object Relational Tutorial (1.x API)

The `SQLAlchemy Object Relational Mapper` presents a method of _associating_ __user-defined `Python classes` with `database tables`__, and _instances_ of those classes (_objects_) with `rows` in their corresponding `tables`. It _includes_ a system that __transparently synchronizes all changes in state__ between _objects_ and their _related rows_, called a `unit of work`, as well as a system for _expressing database queries_ in terms of the `user defined classes` and their `defined relationships` between each other.

The `ORM` is in _contrast_ to the `SQLAlchemy Expression Language`, upon which the `ORM` is constructed. Whereas the `SQL Expression Language`, introduced in `SQL Expression Language Tutorial (1.x API)`, presents a system of _representing_ the __primitive constructs of the relational database directly__ _without opinion_, the `ORM` presents a __high level__ and __abstracted__ `pattern of usage`, which itself is an example of applied usage of the _Expression Language_.

While there is __overlap__ among the `usage patterns` of the `ORM` and the `Expression Language`, the __similarities are more superficial__ than they may at first appear. One approaches the _structure and content of data_ from the `perspective` of a __`user-defined domain model`__ which is __transparently persisted and refreshed__ from its _underlying storage model_. The other approaches it from the perspective of __`literal schema` and `SQL expression` representations__ which are __explicitly composed into messages__ _consumed individually by the database_.

A _successful_ `application` may be constructed using the `Object Relational Mapper` __exclusively__. In _advanced situations_, an application constructed with the ORM may make _occasional usage_ of the `Expression Language` __directly__ in certain areas where __specific database interactions are required__.

#### Version Check

A _quick check_ to __verify__ that we are on at `least version 1.4` of `SQLAlchemy`.

In [3]:
import sqlalchemy
from sqlalchemy import create_engine

In [2]:
sqlalchemy.__version__

'1.4.41'

#### Connecting

For this notebook we will use an __in-memory-only__ `SQLite database`. To connect we use `create_engine()`.

In [4]:
engine = create_engine("sqlite:///:memory:", echo=True)

The `echo` flag is a _shortcut_ to __setting up `SQLAlchemy` logging__, which is accomplished via __Python's standard `logging` module__. With it enabled, we'll see _all the generated SQL produced_. If you are working through this notebook and want _less output generated_, set it to `False`. This notebook will _format_ the `SQL` __behind a popup window__ so it doesn't get in our way; just click the `"SQL"` links to see what's being _generated_.

The _return value_ of `create_engine()` is an __instance of `Engine`__, and it _represents_ the __core interface to the database__, adapted through a `dialect` that _handles the details of the database and DBAPI in use_. In this case the `SQLite dialect` will __interpret instructions__ to the Python _built-in_ `sqlite3` module.

> ##### Lazy Connecting
> The `Engine`, when first returned by `create_engine()`, has __not actually tried to connect__ to the database yet; that _happens only the first time it is asked to perform a task_ against the database.

The first time a method like `Engine.execute()` or `Engine.connect()` is called, the `Engine` _establishes_ a __real `DBAPI` connection__ to the database, which is then __used to emit the SQL__. When using the `ORM`, we typically __don't use the `Engine` directly once created__; instead, it's _used behind the scenes_ by the `ORM` as we'll see shortly.